In [2]:
import os
import pandas as pd
pd.set_option("display.max_colwidth", None)

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://dalle3-swo.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "e51119f8d8774069a6594d92ccf7a70d"

In [3]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_core.output_parsers import BaseOutputParser
from typing import List, Optional
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo",
    temperature=0
) 

llm_16k = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo-16k",
    temperature=0
) 

smart_llm = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-4",
    temperature=0
) 
gpt_35_turbo_instruct = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo-instruct",
    temperature=0
) 
# Embedding
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2024-02-15-preview",
)

# Question list

In [ ]:
"I'm at BUV. I want to go to 87 Lang Ha, tell me available bus",
"I'm at BUV. I want to go to AEON Mall, tell me available bus",
"Tell me what are available routes for Monday 27 May",
"What is the last bus timing from 87 LANG HA to BUV CAMPUS? I have a study group that ends late.",
"How do I get updated information about daily traffic situations for the CAU GIAY route?"

In [43]:
question_list = [
    "What time does the bus start on 28 May?",
    "When does the bus arrive at Sky oasis?",
    "What is the starting time of the bus from Le Dai Hanh on 27 May?",
    "On which days does the bus operate in Times city?",
    "What are the pickup points available in Hai Ba Trung on 28 May?",
    "Can you provide the bus schedule for 28 May"
]

# Error
- Wrong query

# load data

In [5]:
filepath = [
    "../dataInput/bus_route/bus_schedule.csv",
    "../dataInput/bus_route/bus_timetable.csv",
    "../dataInput/bus_route/bus_trips.csv",
]
# list_of_dfs = [pd.read_csv(file) for file in filepath]
df_bus_schedule = pd.read_csv(filepath[0])
df_bus_timetable = pd.read_csv(filepath[1])
df_bus_trips = pd.read_csv(filepath[2])

In [6]:
df_bus_schedule.head()

,trip_id,stop_sequence,stop_name,stop_time
0,1,1,Le Dai Hanh,07:40:00
1,1,2,Time City,07:55:00
2,1,3,Aeon Mall Long Bien,08:05:00
3,1,4,Rung Co - E2 Building,08:20:00
4,1,5,BUV Campus,08:25:00


In [7]:
df_bus_timetable.head()

,day_of_week,trip_id,date
0,Monday,1,27/05/2024
1,Monday,2,27/05/2024
2,Monday,3,27/05/2024
3,Monday,4,27/05/2024
4,Tuesday,1,28/05/2024


# CSV Agent

In [8]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent, create_pandas_dataframe_agent
custom_prefix = """
    You are an intelligent assistant designed to help users interact with bus trip data. The data is 
    organized into three main tables: 
    1. bus_trips.csv: Contains information about bus trips (departure time, arrival time, route, etc.). 
    2. bus_schedule.csv: Contains detailed information about the schedule of each bus trip (stops, 
    stop times, etc.). 
    stop_sequence=1 means this is first pickup point,
    highest stop_sequence means this is drop-off point.
    3. bustimetable.csv: Contains information about the timetable of bus trips for different days of the week. 

    Your answer must strictly follow this rules: 
    Rule 1: your answer must provide information as nature language, not as a table or dataframe, 
    list down as different bullets to make your answer readable
    Rule 2: don't mention anything about code, dataframe or something like that in your answer 
    Rule 3: if unclear question, say that you don't know this information 
    Rule 4: don't try to answer unavailable information 

    Please provide clear and concise responses based on the data available in these tables
"""
custom_suffix = ""

list_of_dfs = [df_bus_schedule, df_bus_timetable, df_bus_trips]

agent = create_pandas_dataframe_agent(
    # replace llm model
    llm_16k,
    df=list_of_dfs,
    prefix=custom_prefix,
    suffix=custom_suffix,
    verbose=True,
    # number_of_head_rows=100,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [ ]:
agent.invoke("")

In [26]:


file_path = "../data/bus_route/Refined_Bus_schedules.csv"
agent = create_csv_agent(
    llm_16k,
    file_path,
    verbose=True,
    # agent_type="tool-calling",
    agent_type=AgentType.OPENAI_FUNCTIONS,
    agent_executor_kwargs = {"handle_parsing_errors": True}
)

In [44]:
question_list

['What time does the bus start on 28 May?',
 'When does the bus arrive at Sky oasis?',
 'What is the starting time of the bus from Le Dai Hanh on 27 May?',
 'On which days does the bus operate in Times city?',
 'What are the pickup points available in Hai Ba Trung on 28 May?',
 'Can you provide the bus schedule for 28 May']

In [50]:
agent.invoke(question_list[5])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Date'] == '28/05/2024']"}`


          Date  Day      District                 Pickup point Starting time  \
44  28/05/2024  Tue  Hai Ba Trung                  Le Dai Hanh      07:40:00   
45  28/05/2024  Tue  Hai Ba Trung                   Times City      07:55:00   
46  28/05/2024  Tue  Hai Ba Trung          AEON Mall Long Bien      08:05:00   
47  28/05/2024  Tue  Hai Ba Trung        Rung Co - E2 Building      08:20:00   
48  28/05/2024  Tue  Hai Ba Trung                   BUV Campus      08:25:00   
49  28/05/2024  Tue  Hai Ba Trung                    Sky Oasis      08:28:00   
50  28/05/2024  Tue  Hai Ba Trung      Sol Forest & Haven Park      08:28:00   
51  28/05/2024  Tue  Hai Ba Trung         Westbay - B Building      08:31:00   
52  28/05/2024  Tue  Hai Ba Trung         Westbay - C Building      08:31:00   
53  28/05/2024  Tue  Hai Ba Trung     Aquabay - Sky 2 Building      08:34:00  

KeyboardInterrupt: 

In [ ]:
import pandas as pd
answer_df = pd.DataFrame({
    "Question": question_list,
    "Bot Answer": bot_answers
})